In [1]:
import io

import pandas as pd
import requests

In [2]:
endpoint = "https://re.jrc.ec.europa.eu/api/v5_2/seriescalc"

In [10]:
params = {
    "lat": 48.778966,
    "lon": 2.291469,
    "startyear": 2006,
    "endyear": 2010,
    "outputformat": "csv",
}

In [11]:
response = requests.get(endpoint, params=params)

In [12]:
if response.status_code == 200:
    data = pd.read_csv(
        io.StringIO(response.content.decode("utf-8")),
        skiprows=8,
        skipfooter=11,
        engine="python",
    )

In [40]:
dataset = data.drop(columns=["H_sun", "Int"])
dataset["Datetime"] = pd.to_datetime(dataset["time"], format="%Y%m%d:%H%M")
dataset = dataset.drop(columns=["time"])

In [41]:
weekly_episodes = dataset.groupby(pd.Grouper(key='Datetime', freq='W'))

In [42]:
weekly_episodes.describe()

G(i)                                                         \
            count       mean         std  min  25%  50%      75%     max   
Datetime                                                                   
2006-01-01   24.0  34.880000   72.717662  0.0  0.0  0.0  22.3175  255.01   
2006-01-08  168.0  36.902619   71.830805  0.0  0.0  0.0  43.0000  308.01   
2006-01-15  168.0  35.196369   74.989958  0.0  0.0  0.0  27.2500  324.01   
2006-01-22  168.0  34.564405   66.893402  0.0  0.0  0.0  49.0000  335.01   
2006-01-29  168.0  62.761250  109.675957  0.0  0.0  0.0  75.2500  386.02   
...           ...        ...         ...  ...  ...  ...      ...     ...   
2010-12-05  168.0  32.449643   61.899531  0.0  0.0  0.0  41.3475  280.01   
2010-12-12  168.0  25.305417   50.518206  0.0  0.0  0.0  35.3025  272.01   
2010-12-19  168.0  37.926071   75.117476  0.0  0.0  0.0  34.5000  303.02   
2010-12-26  168.0  30.232857   58.305278  0.0  0.0  0.0  33.3150  243.01   
2011-01-02  119.0  30.079580   56.859557  0.0  0.0  0.0  41.0600  240.01   

              T2m            ...                 WS10m                      \
            count      mean  ...     75%    max  count      mean       std   
Datetime                     ...                                             
2006-01-01   24.0  4.544167  ...  5.8125   6.98   24.0  4.336250  0.368201   
2006-01-08  168.0  1.832679  ...  3.2875   6.00  168.0  2.354524  0.731199   
2006-01-15  168.0  3.117321  ...  4.8150   7.32  168.0  2.600595  0.725749   
2006-01-22  168.0  5.994643  ...  7.7975  10.21  168.0  3.126548  1.197615   
2006-01-29  168.0 -0.576369  ...  1.1550   5.69  168.0  3.245298  1.071590   
...           ...       ...  ...     ...    ...    ...       ...       ...   
2010-12-05  168.0 -1.200179  ...  0.0200   3.05  168.0  3.071488  1.392111   
2010-12-12  168.0  1.327083  ...  2.8300   6.05  168.0  2.836905  0.745992   
2010-12-19  168.0 -0.109107  ...  1.4775   5.27  168.0  3.668750  1.543482   
2010-12-26  168.0 -0.380298  ...  1.6825   4.25  168.0  3.401012  1.715232   
2011-01-02  119.0  1.090504  ...  2.8150   5.20  119.0  2.533445  0.685802   

                                               
             min     25%    50%     75%   max  
Datetime                                       
2006-01-01  3.79  4.1050  4.280  4.5675  5.24  
2006-01-08  0.69  1.7725  2.375  2.9000  3.93  
2006-01-15  0.14  2.2100  2.760  3.0300  4.76  
2006-01-22  0.62  2.3925  2.970  3.6075  6.83  
2006-01-29  0.83  2.5500  3.240  3.9475  6.48  
...          ...     ...    ...     ...   ...  
2010-12-05  0.14  2.0525  3.065  4.0875  5.86  
2010-12-12  1.03  2.4100  2.830  3.1700  5.72  
2010-12-19  0.69  2.6725  3.380  4.2950  9.17  
2010-12-26  0.21  1.9825  3.030  5.1700  6.97  
2011-01-02  1.03  2.0700  2.620  3.0300  4.07  

[262 rows x 24 columns]

In [44]:
with pd.HDFStore('./datasets/weather.h5') as store:
    # Save each DataFrame with a key
    for i, (name, df) in enumerate(weekly_episodes):
        store[f'eps_{i}'] = df